In [46]:
import numpy as np
import pandas
import scipy
import data as dt
import statsmodels.tsa.api as sm
from statsmodels.tsa.vecm.vecm import VECM

Data import
===========

In [2]:
iidata = dt.load_pandas()

In [3]:
mdata = iidata.data

dates = mdata[['year', 'quarter']].astype(int).astype(str)

quarterly = dates["year"] + "Q" + dates["quarter"]z

from statsmodels.tsa.base.datetools import dates_from_str

quarterly = dates_from_str(quarterly)

mdata = mdata[['Dp','R']]

mdata.index = pandas.DatetimeIndex(quarterly)

data = mdata

In [4]:
model = VECM(data)

In [5]:
results = model.fit(2)

fit called
2


In [6]:
results==None

True

In [7]:
type(data)                     # data is what VECM gets. 

pandas.core.frame.DataFrame

LS estimator
============

In [213]:
# just trying out the construction of the part containing dummy variables
seasons = np.zeros((3, 12)) # 3 = 4-1 (4=number_of_seasons)
for i in range(seasons.shape[0]):
    seasons[i, i::4] = i+1 # dummy variables
seasons

array([[ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  2.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  2.,  0.,  0.],
       [ 0.,  0.,  3.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  3.,  0.]])

In [8]:
deterministic_terms = '' # \in \{'c', 'lt', 's'\} c=constant, lt=linear trend, s=seasonal terms
'c' in deterministic_terms

False

In [9]:
y_all = data.values.T;   y_all.shape

(2, 107)

In [10]:
diff_lags = 3       # p-1
p = diff_lags + 1   # lags / size of needed presample
K = y_all.shape[0]

In [11]:
y_1_T = y_all[:, p:];   y_1_T.shape

(2, 103)

In [12]:
T = y_1_T.shape[1]

In [13]:
Delta_y_all = np.diff(y_all);   Delta_y_all.shape

(2, 106)

In [14]:
Delta_y_1_T = Delta_y_all[:, p-1:];   Delta_y_1_T.shape

(2, 103)

In [15]:
y_minus1 = y_all[:, p-1:-1]
if 'lt' in deterministic_terms:
    y_minus1 = np.vstack((y_minus1, np.arange(T)))
y_minus1.shape

(2, 103)

In [16]:
Delta_x = np.zeros((diff_lags*K, T));   Delta_x.shape

(6, 103)

In [17]:
for j in range(Delta_x.shape[1]):
    Delta_x[:, j] = Delta_y_all[:, j+p-2:None if j-1<0 else j-1:-1].T.reshape(K*(p-1))
if 'c' in deterministic_terms:
    Delta_x = np.vstack((Delta_x, np.ones(T)))

In [18]:
mat1 = np.hstack((Delta_y_1_T.dot(y_minus1.T), Delta_y_1_T.dot(Delta_x.T)))

In [19]:
B = y_minus1.dot(Delta_x.T)
mat2 = np.linalg.inv(np.vstack( (np.hstack( (y_minus1.dot(y_minus1.T),   B                    ) ),
                                 np.hstack( (B.T,                        Delta_x.dot(Delta_x.T) )) ) ))

In [20]:
estPiGamma = mat1.dot(mat2)
estPiGamma.round(2)

array([[-0.54,  0.06, -0.59,  0.12, -0.71,  0.19, -0.83,  0.02],
       [ 0.13, -0.02, -0.1 ,  0.18, -0.05, -0.04,  0.  ,  0.17]])

In [21]:
Pi_hat_cols = K if 'lt' not in deterministic_terms else K+1
Pi_hat = estPiGamma[:, :Pi_hat_cols]
Pi_hat

array([[-0.54327263,  0.05905934],
       [ 0.12695865, -0.02184917]])

In [22]:
Gamma_hat = estPiGamma[:, Pi_hat_cols:]
np.round(Gamma_hat, 2)

array([[-0.59,  0.12, -0.71,  0.19, -0.83,  0.02],
       [-0.1 ,  0.18, -0.05, -0.04,  0.  ,  0.17]])

In [23]:
_A = Delta_y_1_T - Pi_hat.dot(y_minus1) - Gamma_hat.dot(Delta_x)
Sigma_u_hat = 1/(T-K*p) * np.dot(_A,_A.T)
Sigma_u_hat

array([[  3.94865227e-05,  -3.82768919e-06],
       [ -3.82768919e-06,   3.03766974e-05]])

EGLS estimator
==============

In [24]:
r = 1 # r will be given as an function argument. r \in \{0, \ldots, K\}. r=0, r=K?
alpha_hat = Pi_hat[:, :r] # LS Gamma_hat has to be calculated before this line
M = np.identity(T) - Delta_x.T.dot(np.linalg.inv(Delta_x.dot(Delta_x.T))).dot(Delta_x)
R_0 = Delta_y_1_T.dot(M)
R_1 = y_minus1.dot(M)
R_11 = R_1[:r]
R_12 = R_1[r:]
_alpha_Sigma = alpha_hat.T.dot(np.linalg.inv(Sigma_u_hat))
beta_hhat = np.linalg.inv(_alpha_Sigma.dot(alpha_hat)).dot(_alpha_Sigma).dot(R_0-alpha_hat.dot(R_11)).dot(R_12.T).dot(np.linalg.inv(R_12.dot(R_12.T)))
beta_hhat = np.vstack((1, beta_hhat))
print(alpha_hat)
print(beta_hhat)
# ? Gamma_hhat necessary / computed via (Delta_y_1_T - alpha_hat.dot(beta_hhat.T).dot(y_minus1)).dot(Delta_x.dot(np.linalg.inv(np.dot(Delta_x,Delta_x.T))))

[[-0.54327263]
 [ 0.12695865]]
[[ 1.        ]
 [-0.11131212]]


ML estimator
============

In [26]:
# r, M R_0, R_1 as for the EGLS estimator
S = np.bmat([[Ri.dot(Rj.T)/T for Rj in [R_0, R_1]] for Ri in [R_0, R_1]])
S

matrix([[  3.93591032e-05,  -4.05327169e-06,  -7.62693213e-06,
          -2.03864203e-05],
        [ -4.05327169e-06,   2.84786061e-05,  -3.61793778e-06,
          -4.21339812e-05],
        [ -7.62693213e-06,  -3.61793778e-06,   8.69896387e-05,
           6.71056510e-04],
        [ -2.03864203e-05,  -4.21339812e-05,   6.71056510e-04,
           5.82770122e-03]])

In [27]:
S00 = S[:K, :K]
S01 = S[:K, K:]
S10 = S[K:, :K]
S11 = S[K:, K:]
S11_ = scipy.linalg.sqrtm(S11.I)

v = np.linalg.eig(S11_ * S10 * S00.I * S01 * S11_)[1]
m_beta_tilde = v.T * S11_
m_alpha_tilde = S01 * m_beta_tilde * (m_beta_tilde.T * S11 * m_beta_tilde).I

m_Delta_y_1_T = np.matrix(Delta_y_1_T)
m_y_minus1 = np.matrix(y_minus1)
m_Delta_x = np.matrix(Delta_x)
m_Gamma_tilde = (m_Delta_y_1_T - m_alpha_tilde*m_beta_tilde.T*m_y_minus1) * m_Delta_x.T * (m_Delta_x * m_Delta_x.T).I
m_temp = (m_Delta_y_1_T - m_alpha_tilde*m_beta_tilde.T*m_y_minus1 - m_Gamma_tilde*m_Delta_x)
m_Sigma_u_tilde = m_temp * m_temp.T / T
print("alpha")
print(m_alpha_tilde)
print("beta")
print(m_beta_tilde)
print("Gamma")
print(m_Gamma_tilde)
print("Sigma_u")
print(m_Sigma_u_tilde)

alpha
[[-0.00172057 -0.00014795]
 [ 0.00033459 -0.00057592]]
beta
[[ 318.7816345   -35.24565026]
 [ -35.7992249    17.13983205]]
Gamma
[[-0.58982483  0.1170476  -0.70501833  0.18765474 -0.82607809  0.02306974]
 [-0.09868298  0.18443278 -0.0515211  -0.04323455  0.0034007   0.17215407]]
Sigma_u
[[  3.64196083e-05  -3.53039294e-06]
 [ -3.53039294e-06   2.80173422e-05]]


LS estimator in one function
============================

In [334]:
def estimation_matrices(data, diff_lags, deterministic_terms):
    p = diff_lags+1
    y_all = data.values.T
    K = y_all.shape[0]
    
    y_1_T = y_all[:, p:]
    T = y_1_T.shape[1]
    
    Delta_y_all = np.diff(y_all)
    Delta_y_1_T = Delta_y_all[:, p-1:]
    
    y_minus1 = y_all[:, p-1:-1]
    if 'lt' in deterministic_terms:
        y_minus1 = np.vstack((y_minus1, np.arange(T)))
    
    Delta_x = np.zeros((diff_lags*K, T))
    for j in range(Delta_x.shape[1]):
        Delta_x[:, j] = Delta_y_all[:, j+p-2:None if j-1<0 else j-1:-1].T.reshape(K*(p-1))
    if 'c' in deterministic_terms:
        Delta_x = np.vstack((Delta_x, np.ones(T)))
    if 's' in deterministic_terms: # TODO: How many seasons??
        seasons = np.zeros((3, Delta_x.shape[1])) # 3 = 4-1 (4=number_of_seasons)
        for i in range(seasons.shape[0]):
            seasons[i, i::4] = i+1
        Delta_x = np.vstack((Delta_x, seasons))
    
    return y_all, y_1_T, Delta_y_all, Delta_y_1_T, y_minus1, Delta_x

In [202]:
def _ls_Pi_Gamma(Delta_y_1_T, y_minus1, Delta_x, deterministic_terms):
    mat1 = np.hstack((Delta_y_1_T.dot(y_minus1.T), Delta_y_1_T.dot(Delta_x.T)))

    B = y_minus1.dot(Delta_x.T)
    mat2 = np.linalg.inv(np.vstack( (np.hstack( (y_minus1.dot(y_minus1.T),   B                    ) ),
                                     np.hstack( (B.T,                        Delta_x.dot(Delta_x.T) )) ) ))

    estPiGamma = mat1.dot(mat2)

    Pi_hat_cols = K if 'lt' not in deterministic_terms else K+1
    Pi_hat = estPiGamma[:, :Pi_hat_cols]

    Gamma_hat = estPiGamma[:, Pi_hat_cols:]
    #print("Dy1T.shape: "+str(Delta_y_1_T.shape) + ",  Pi_hat.shape: "+str(Pi_hat.shape) + ",  Gamma_hat.shape: "+str(Gamma_hat.shape))
    #print("yminus1:" + str(y_minus1.shape))
    _A = Delta_y_1_T - Pi_hat.dot(y_minus1) - Gamma_hat.dot(Delta_x)
    p = diff_lags+1
    Sigma_u_hat = 1/(T-K*p) * np.dot(_A,_A.T)

    return (Pi_hat, Gamma_hat, Sigma_u_hat)    

In [203]:
def ls_estimator(data, diff_lags, deterministic_terms):    
    # deterministic_terms \in \{'c', 'lt', 's'\} c=constant, lt=linear trend, s=seasonal terms

    y_all, y_1_T, Delta_y_all, Delta_y_1_T, y_minus1, Delta_x = estimation_matrices(data, diff_lags, deterministic_terms)
    K = y_all.shape[0]
    T = y_1_T.shape[1]

    Pi_hat, Gamma_hat, Sigma_u_hat = _ls_Pi_Gamma(Delta_y_1_T, y_minus1, Delta_x, deterministic_terms)
    
    print("Pi:")
    print(Pi_hat)
    print("Gamma:")
    print(Gamma_hat)
    print("Sigma_u")
    print(Sigma_u_hat)
    
    return {"Pi_hat": Pi_hat, "Gamma_hat": Gamma_hat, "Sigma_u_hat": Sigma_u_hat}

In [209]:
ls_estimator(data, 3, 's')

Pi:
[[-0.36278647 -0.03297439]
 [ 0.25397334 -0.09315322]]
Gamma:
[[ -7.43193207e-01   2.03622004e-01  -7.77866051e-01   1.63307948e-01
   -6.09499930e-01   1.33131557e-01   1.41883529e-03   6.79750209e-04
    6.47776643e-03]
 [ -3.35996508e-02   2.51145907e-01  -7.05307414e-02  -3.99504228e-02
   -9.72528296e-03   2.26631612e-01   1.18210692e-02   1.50453090e-03
    9.74824196e-04]]
Sigma_u
[[  3.25768936e-05  -4.26797119e-06]
 [ -4.26797119e-06   2.79369870e-05]]


{'Gamma_hat': array([[ -7.43193207e-01,   2.03622004e-01,  -7.77866051e-01,
           1.63307948e-01,  -6.09499930e-01,   1.33131557e-01,
           1.41883529e-03,   6.79750209e-04,   6.47776643e-03],
        [ -3.35996508e-02,   2.51145907e-01,  -7.05307414e-02,
          -3.99504228e-02,  -9.72528296e-03,   2.26631612e-01,
           1.18210692e-02,   1.50453090e-03,   9.74824196e-04]]),
 'Pi_hat': array([[-0.36278647, -0.03297439],
        [ 0.25397334, -0.09315322]]),
 'Sigma_u_hat': array([[  3.25768936e-05,  -4.26797119e-06],
        [ -4.26797119e-06,   2.79369870e-05]])}

EGLS estimator in one function
==============================

In [230]:
def _M_and_R_matrices(T, Delta_x, Delta_y_1_T, y_minus1):
    M = np.identity(T) - Delta_x.T.dot(np.linalg.inv(Delta_x.dot(Delta_x.T))).dot(Delta_x)
    R_0 = Delta_y_1_T.dot(M)
    R_1 = y_minus1.dot(M)
    return (M, R_0, R_1)

In [233]:
def egls_estimator(data, diff_lags, deterministic_terms='', r=1):    
    _ya, y_1_T, _Dya, Delta_y_1_T, y_minus1, Delta_x = estimation_matrices(data, diff_lags, deterministic_terms)
    K = y_all.shape[0]
    T = y_1_T.shape[1]
    
    Pi_hat, _Gamma_hat, Sigma_u_hat = _ls_Pi_Gamma(Delta_y_1_T, y_minus1, Delta_x, deterministic_terms)
    #print("Dy1T.shape: "+str(Delta_y_1_T.shape) + ",  Pi_hat.shape: "+str(Pi_hat.shape) + ",  Gamma_hat.shape: "+str(Gamma_hat.shape))
    #print("yminus1: " + str(y_minus1.shape) + ",  DX.shape: "+str(Delta_x.shape))
    alpha_hat = Pi_hat[:, :r]
    
    M, R_0, R_1 = _M_and_R_matrices(T, Delta_x, Delta_y_1_T, y_minus1)
    R_11 = R_1[:r]
    R_12 = R_1[r:]
    #print("Sigma_u_hat: \n"+str(Sigma_u_hat))
    _alpha_Sigma = alpha_hat.T.dot(np.linalg.inv(Sigma_u_hat))
    beta_hhat = np.linalg.inv(_alpha_Sigma.dot(alpha_hat)).dot(_alpha_Sigma).dot(R_0-alpha_hat.dot(R_11)).dot(R_12.T).dot(np.linalg.inv(R_12.dot(R_12.T))).T
    beta_hhat = np.vstack((1, beta_hhat))
    # ? Gamma_hhat necessary / computed via (Delta_y_1_T - alpha_hat.dot(beta_hhat.T).dot(y_minus1)).dot(Delta_x.dot(np.linalg.inv(np.dot(Delta_x,Delta_x.T))))
    
    print("alpha")
    print(alpha_hat.round(3))
    print("beta")
    print(beta_hhat.round(3))

In [234]:
egls_estimator(data, 3, 'cs')

alpha
[[-0.661]
 [ 0.341]]
beta
[[ 1.   ]
 [-0.243]]


ML estimator in one function
============================

In [322]:
def ml_estimator(data, diff_lags, deterministic_terms='', r=1):
    _ya, y_1_T, _Dya, Delta_y_1_T, y_minus1, Delta_x = estimation_matrices(data, diff_lags, deterministic_terms)
    K = y_all.shape[0]
    T = y_1_T.shape[1]
    
    M, R_0, R_1 = _M_and_R_matrices(T, Delta_x, Delta_y_1_T, y_minus1)
    S = np.bmat([[Ri.dot(Rj.T)/T for Rj in [R_0, R_1]] for Ri in [R_0, R_1]])
    S00 = S[:K, :K]
    S01 = S[:K, K:]
    S10 = S[K:, :K]
    S11 = S[K:, K:]
    S11_ = scipy.linalg.sqrtm(S11.I)

    v = np.linalg.eig(S11_ * S10 * S00.I * S01 * S11_)[1]
    #print(v)
    m_beta_tilde = (v[:,:r].T * S11_).T
    m_beta_tilde = m_beta_tilde * m_beta_tilde[:r,:r].I # normalize such that eye(r) forms the first r rows of beta tilde
    #print(m_beta_tilde.T)
    m_alpha_tilde = S01 * m_beta_tilde * (m_beta_tilde.T * S11 * m_beta_tilde).I

    m_Delta_y_1_T = np.matrix(Delta_y_1_T)
    m_y_minus1 = np.matrix(y_minus1)
    m_Delta_x = np.matrix(Delta_x)
    m_Gamma_tilde = (m_Delta_y_1_T - m_alpha_tilde*m_beta_tilde.T*m_y_minus1) * m_Delta_x.T * (m_Delta_x * m_Delta_x.T).I
    m_temp = (m_Delta_y_1_T - m_alpha_tilde*m_beta_tilde.T*m_y_minus1 - m_Gamma_tilde*m_Delta_x)
    m_Sigma_u_tilde = m_temp * m_temp.T / T
    print("alpha")
    print(m_alpha_tilde)
    print("beta")
    print(m_beta_tilde)
    print("==> Pi")
    print(m_alpha_tilde*m_beta_tilde.T)
    print("Gamma")
    print(m_Gamma_tilde)
    print("Sigma_u")
    print(m_Sigma_u_tilde)

In [335]:
ml_estimator(data, 3, 's')

alpha
[[-0.16926027]
 [ 0.32080276]]
beta
[[ 1.        ]
 [-0.21559865]]
==> Pi
[[-0.16926027  0.03649229]
 [ 0.32080276 -0.06916464]]
Gamma
[[ -1.00547521e+00   1.82864810e-01  -1.01860250e+00   1.55046265e-01
   -7.74745204e-01   7.04048739e-02  -7.19009291e-03  -4.03917869e-03
    3.60199040e-03]
 [ -1.24172170e-01   2.43977930e-01  -1.53663044e-01  -4.28033881e-02
   -6.67885981e-02   2.04970523e-01   8.84819152e-03  -1.25032963e-04
   -1.82530013e-05]]
Sigma_u
[[  3.42146933e-05  -2.49714876e-06]
 [ -2.49714876e-06   2.62641603e-05]]
